<a href="https://colab.research.google.com/github/Capou/FishClassifier/blob/master/fish_classifier_backup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas 

In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import *
from pandas import *

# The Amazing Fish Classifier!

You need to know whether you're being chased by a dangerous grizzly, or a sweet teddy bear, and you need an answer *fast*? Then you've come to the right place. Take a pic of the potentially vicious killer, and click 'upload' to classify it. (Important: this only handles grizzly bears, black bears, and teddy bears. It will **not** give a sensible answer for polar bears, a bear market, a bear of a man, or hot dogs.

----

In [ ]:
path = Path()
learn_inf = load_learner(path/'export-059.pkl', cpu=True)
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
out_pl_df = widgets.Output()
lbl_pred = widgets.Label()

/home/jhoward/anaconda3/lib/python3.7/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/jhoward/anaconda3/lib/python3.7/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/jhoward/anaconda3/lib/python3.7/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by ac

UnpicklingError: invalid load key, '\x0a'.

In [ ]:
def create_df(): 
    fish = {'Familie': ['Leuciscidae', 'Icaluridae', 'Cobitidae', 'Cyprinidae', 'Esocidae', 'Percidae', 'Salmonidae'],
        'Spezies': ['abramis_brama', 'ameiurus_nebulosus', 'cobitis_taenia', 'cyprinus_carpio', 'esox_lucius', 'gymnocephalus_cernua', 'salmo_trutta'],
        'Name': ['Brasse', 'Katzenwels', 'Dorngrundel', 'Karpfen', 'Hecht', 'Kaulbarsch', 'Forelle'],
        'Koerperform':['Hochrueckig', 
                       'bauch abgeplattet', 
                       'Spindel-/Torpedoform', 
                       'langgestreckt flachrueckig, walzenfoerming', 
                       'Pfeilfoermig', 
                       'seitlich abgeflacht, leicht hochrueckig', 
                       'spindel-, torpedofoermig'],
        'Mund': ['Ruesselmaul', 
                 'endständig', 
                 'unterstaendig', 
                 'endstaendig, vorstuelpbar, ruesselmaul', 
                 'oberstaendig, entenschnabelform, weite Maulspalte', 
                 'endstaendig', 
                 'endstaendig, Maulspalte reicht bis hinter die Augen, Leichhaken'],
        'Schuppen': ['realtiv groß', 
                     'keine', 
                     'sehr klein und duenn', 
                     'voll beschuppt', 
                     'klein, voll beschuppt', 
                     'klein', 
                     'kleine Rundschuppen'],
        'Farbe': ['alle Flossen dunkelgrau', 
                  'variabel', 
                  'Koerper mit laengsreihen dunkler Punkte', 
                  'Ruecken: bronze-gelb; Schwanz: roetlich', 
                  'Flossen mit dunklen Flecken, Seiten mit dunklen Querbinden', 
                  'Garu- und Gruentoene, dunkle Flecken auf Flossen', 
                  'Flanken mit schwarzen Flecken und Punkten'], 
        'Rueckenflosse': ['beginn hinter Bauchflossenansatz', 
                          'beginnt vor Bauchflossenansatz', 
                          'auffallend hoch, beginnt vor Bauchflossenansatz', 
                          '-', 
                          'weit nach hinten versetzt, Hinterrand stark gerundet',
                          'zwei, miteinander verbunden',
                          'beginnt vor Bauchlossenansatz'], 
        'Schwanzflosse': ['tief gegabelt, unsymmetrisch',
                          'gleichmäßig - gerundet',
                          'leicht gerundet',
                          '-',
                          'gegabelt',
                          'gleichmaeßig',
                          'bei erwachsenen Fisch am Hinterrand fast gerade'],
        'Afterflosse':['sehr lang, konkav',
                       'groß, konvex',
                       'konvex',
                       '-',
                       'konvex',
                       'Flossenhaut zwischen vorderen Stachelstrahl wenig eingebuchtet',
                       '-'], 
        'Brust/Bauchflosse':['reichen bis Bauchflossenansatz',
                       '-', 
                       'Bauchflosse bauchstaendig',
                       '-',
                       'Bauchflosse bauchstaendig',
                       '-',
                       '-'], 
        'Besonderheiten':['Augendurchmesser im Verhaeltnis zur Schnauzenlaenge kleiner',
                          '8 Barteln',
                          '6 kurze Barteln, Oberlippe',
                          '4 Barteln, Oberlippe',
                          'kraeftige Fangzaehne, Seitenlinie mehrfach unterbrochen',
                          'Kiemendeckel spitz auslaufend (Dorn)',
                          'Fettflosse, können nicht am gedrungenen Schwanzstiel gehalten werden (im vgl. zum Lachs) '], 
        }
    df = pandas.DataFrame(fish, columns = ['Familie', 'Spezies', 'Name', 'Koerperform', 'Mund', 'Schuppen', 'Farbe', 'Rueckenflosse', 'Schwanzflosse', 'Afterflosse', 'Brust/Bauchflosse','Besonderheiten'], index=['abramis_brama', 'ameiurus_nebulosus', 'cobitis_taenia', 'cyprinus_carpio', 'esox_lucius', 'gymnocephalus_cernua', 'salmo_trutta'])
    return df 

In [ ]:
def on_data_change(change):
    lbl_pred.value = ''
    df = create_df()
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    out_pl_df.clear_output()
    with out_pl_df: display(df.loc[f'{pred}'])
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [ ]:
btn_upload.observe(on_data_change, names=['data'])

In [ ]:
display(HBox([VBox([widgets.Label('Select your bear!'), btn_upload, out_pl, lbl_pred]), VBox([widgets.Label('Informationen'), out_pl_df])))